## Training process comparison

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Define a function to plot the evolution of the specified metric (mean, standard deviation, max)
def plot_fitness_evolution(ax1, EA_variable, fitness_mode, enemy, metric='fitness_values', fitness_function=''):
    base_folder = 'Group_96_data_for_plots'
    algorithm_folder = f"{base_folder}/{EA_variable}"
    fitness_folder = f"{algorithm_folder}/fitness_mode_{fitness_mode}"

    # Check if the folder exists
    if not os.path.exists(fitness_folder):
        raise FileNotFoundError(f"Folder {fitness_folder} does not exist.")
    
    # Store data for all run_ids across generations
    generations_data = [[] for _ in range(30)]  # Assuming 30 generations, adjust as needed

    # Loop through run1 to run10, reading data for each generation in each experiment
    for run_id in range(1, 11):  # Assuming 10 experiments
        # Now the run folder includes the enemy number
        run_folder = f"{fitness_folder}/enemy_{enemy}/run{run_id}"  # Updated directory for each enemy
        for generation in range(30):  # Assuming 30 generations
            gen_folder = f"{run_folder}/generation_{generation}_evaluation_enemy_{enemy}"
            metric_file = f"{gen_folder}/{metric}.txt"
            
            if not os.path.exists(metric_file):
                continue  # Skip this generation if the file does not exist
            
            # Read the metric data for each generation
            data = np.loadtxt(metric_file)
            generations_data[generation].append(data)
    
    # Prepare to store the mean and standard deviation for each generation
    means = []
    mean_stds = []  # Standard deviation for Mean value
    max_means = []
    max_stds = []  # Standard deviation for Max value

    # Compute the mean, standard deviation, and mean of the max for each generation
    for generation_data in generations_data:
        # Concatenate all run_id data into one array
        generation_data_flat = np.concatenate(generation_data) if generation_data else np.array([])

        if generation_data_flat.size > 0:
            means.append(np.mean(generation_data_flat))
            mean_stds.append(np.std(generation_data_flat))  # Standard deviation of the Mean
            max_values = [np.max(run_data) for run_data in generation_data]  # Max values per run
            max_means.append(np.mean(max_values))
            max_stds.append(np.std(max_values))  # Standard deviation of the Max
        else:
            means.append(np.nan)
            mean_stds.append(np.nan)
            max_means.append(np.nan)
            max_stds.append(np.nan)

    # Plot Mean value with standard deviation as shaded region
    ax1.plot(range(30), means, marker='o', label=f'Mean value ({EA_variable})')
    ax1.fill_between(range(30), np.array(means) - 0.2 * np.array(mean_stds), np.array(means) + 0.2 * np.array(mean_stds), alpha=0.2)

    # Plot Mean of Max value with standard deviation as shaded region
    ax1.plot(range(30), max_means, marker='x', label=f'Mean of Max value ({EA_variable})')
    ax1.fill_between(range(30), np.array(max_means) - 0.5 * np.array(max_stds), np.array(max_means) + 0.5 * np.array(max_stds), alpha=0.2)

# Example call
enemies = [3, 4, 5]
metric='fitness_values'
fitness_mode = 2
fitness_function = "Offensive" if fitness_mode == 1 else "Survival"

# Create a figure with 3 subplots (one for each enemy)
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 18))  # 3 rows, 1 column

# Loop through the enemies and create a plot for each
for i, enemy in enumerate(enemies):
    ax1 = axes[i]  # Select the appropriate subplot
    ax1.set_xlabel('Generations', fontsize=14)
    ax1.set_ylabel('Fitness value', fontsize=14)
    ax1.set_ylim(-50, 100)

    # Plot for EA_variable="GA"
    plot_fitness_evolution(ax1, EA_variable="GA", fitness_mode=fitness_mode, enemy=enemy, metric='fitness_values', fitness_function=fitness_function)

    # Plot for EA_variable="DE"
    plot_fitness_evolution(ax1, EA_variable="DE", fitness_mode=fitness_mode, enemy=enemy, metric='fitness_values', fitness_function=fitness_function)

    # Set the plot title for each subplot
    ax1.set_title(f'Training process comparison (Fitness Function: {fitness_function}, Enemy: {enemy})', fontsize=16, pad=20)

    # Add legends in the best position for each subplot
    ax1.legend(loc='best', frameon=False, fontsize=12)

# Adjust layout to prevent overlap
plt.tight_layout(rect=[0, 0, 1, 0.98])  # Adjust layout to prevent overlap

# Save the whole figure as one large image
file_name = f"Comparison_for_GA_DE_Fitness_{fitness_function}_All_Enemies".replace(' ', '_').replace(':', '')
plt.savefig(f'Group_96_plots/{file_name}.png', bbox_inches='tight')

# Show the plot
plt.show()


## Boxplot comparing Fitness functions

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import ttest_ind
from evoman.environment import Environment
from demo_controller import player_controller

# Set the number of hidden layer neurons
n_hidden_neurons = 10

headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy" 
    
# Load neural network weights
def load_best_individual(EA_variable, fitness_mode, enemy, run):
    base_folder = 'Group_96_data_for_plots'
    file_path = f"{base_folder}/{EA_variable}/fitness_mode_{fitness_mode}/enemy_{enemy}/run{run}/best_individual.txt"
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist.")
    
    return np.loadtxt(file_path)

# Simulate the neural network and return individual gain
def simulate_best_individual(EA_variable, fitness_mode, enemy, run):
    weights = load_best_individual(EA_variable, fitness_mode, enemy, run)
    experiment_name = "best_individual_simulation"
    
    if not os.path.exists(experiment_name):
        os.makedirs(experiment_name)
    
    env = Environment(
        experiment_name=experiment_name,
        enemies=[enemy],
        playermode="ai",
        player_controller=player_controller(n_hidden_neurons),
        enemymode="static",
        level=2,
        speed="fastest",
        visuals=False,
        randomini="yes"
    )
    
    fitness, player_energy, enemy_energy, time = env.play(pcont=weights)
    individual_gain = player_energy - enemy_energy
    return individual_gain

# Perform simulation and record results
def simulate_for_enemies(EA_variable, fitness_mode, runs=10, enemies=[3, 4, 5]):
    results = {}
    for enemy in enemies:
        enemy_results = []
        for run in range(1, runs + 1):
            for i in range(5):
                individual_gain = simulate_best_individual(EA_variable, fitness_mode, enemy, run)
                enemy_results.append(individual_gain)
        results[enemy] = enemy_results
    return results

# Simulate results for all combinations and store them in all_results
def simulate_for_all_conditions(EA_variables, fitness_modes, runs=10, enemies=[3, 4, 5]):
    all_results = {}
    for EA_variable in EA_variables:
        for fitness_mode in fitness_modes:
            result_key = (EA_variable, fitness_mode)
            all_results[result_key] = simulate_for_enemies(EA_variable, fitness_mode, runs, enemies)
    return all_results

# Example call: simulate all combinations
EA_variables = ["DE", "GA"]  # Assuming two evolutionary algorithms
fitness_modes = [1, 2]  # Two fitness functions

# Run simulation and store all results
all_results = simulate_for_all_conditions(EA_variables, fitness_modes, runs=10, enemies=[3, 4, 5])


### plot

In [ ]:
# Calculate significance level (p-value) and plot the boxplot
def plot_boxplot_with_significance(all_results, EA_variables, enemies=[3, 4, 5]):
    # Collect all data for plotting
    data = []
    labels = []
    color_palette = sns.color_palette("muted", 2)  # Assign different colors for FF1 and FF2
    
    for enemy in enemies:
        for (EA_variable, fitness_mode), result in all_results.items():
            gains = result[enemy]
            ff_label = "FF1" if fitness_mode == 1 else "FF2"
            labels.append(f"{EA_variable} {ff_label} E{enemy}")
            data.append(gains)

    # Create DataFrame for plotting
    df = pd.DataFrame({
        'Individual Gain': np.concatenate(data),
        'Group': np.concatenate([[label] * len(gains) for label, gains in zip(labels, data)])
    })
    
    plt.figure(figsize=(12, 8))
    
    # Assign colors based on fitness function, FF1 uses the first color, FF2 uses the second color
    box_colors = [color_palette[0] if 'FF1' in label else color_palette[1] for label in df['Group']]
    
    # Plot the boxplot using the custom colors
    ax = sns.boxplot(x='Group', y='Individual Gain', data=df, palette=color_palette)
    
    plt.title('Comparison of Individual Gains across EA, Fitness Functions, and Enemies', fontsize=16)
    file_name = 'Comparison of Individual Gains across EA, Fitness Functions, and Enemies'
    plt.xticks(rotation=45)

    # Add a custom legend to differentiate FF1 and FF2
    handles = [
        plt.Line2D([0], [0], color=color_palette[0], lw=4, label='FF1 (Offensive Fitness)'),
        plt.Line2D([0], [0], color=color_palette[1], lw=4, label='FF2 (Survival Fitness)'),
    ]
    plt.legend(handles=handles, title="Fitness Function", loc="best")  # Use loc="best" for automatic placement
    
    # Only compare the differences between different fitness modes for the same EA and same enemy
    unique_groups = df['Group'].unique()
    y_max = df['Individual Gain'].max()  # For positioning the p-value display

    for enemy in enemies:
        for EA_variable in EA_variables:
            group1_label = f"{EA_variable} FF1 E{enemy}"
            group2_label = f"{EA_variable} FF2 E{enemy}"

            if group1_label in unique_groups and group2_label in unique_groups:
                group1 = df[df['Group'] == group1_label]['Individual Gain']
                group2 = df[df['Group'] == group2_label]['Individual Gain']
                
                # Calculate p-value using t-test
                t_stat, p_value = ttest_ind(group1, group2)

                # Set the x-axis positions of the two groups in the plot
                x1, x2 = list(unique_groups).index(group1_label), list(unique_groups).index(group2_label)
                y, h = y_max + 10, 5  # Set the y-axis position and height
                
                # Draw a connecting line between the two boxplots and display the p-value
                ax.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.5, color='black')
                ax.text((x1 + x2) * 0.5, y + h, f"p = {p_value:.3f}", ha='center', va='bottom', color='black')
    plt.savefig(f'Group_96_plots/{file_name}.png', bbox_inches='tight')
    plt.show()

# Example call: Plot the boxplot with significance levels
plot_boxplot_with_significance(all_results, EA_variables)


## Comparing EA

In [ ]:
# 计算显著性水平（p值）并绘制 boxplot
def plot_boxplot_with_significance(all_results, EA_variables, enemies=[3, 4, 5]):
    # 收集所有数据以便绘图
    data = []
    labels = []
    
    # 手动指定颜色：DE为红色，GA为绿色
    color_map = {'DE': 'red', 'GA': 'green'}
    
    for enemy in enemies:
        for (EA_variable, fitness_mode), result in all_results.items():
            # 只使用 FF1 (Fitness Mode 1)
            if fitness_mode == 1:
                gains = result[enemy]
                labels.append(f"{EA_variable} E{enemy}")
                data.append(gains)

    # 创建 DataFrame 用于绘图
    df = pd.DataFrame({
        'Individual Gain': np.concatenate(data),
        'Group': np.concatenate([[label] * len(gains) for label, gains in zip(labels, data)])
    })

    plt.figure(figsize=(12, 8))
    
    # 绘制boxplot
    ax = sns.boxplot(x='Group', y='Individual Gain', data=df, palette=['red', 'green'])

    # 获取box的位置
    box = ax.artists

    # 按组设置每个box的颜色
    for i, artist in enumerate(box):
        group = df['Group'].unique()[i].split()[0]  # 获取组名中的EA
        artist.set_facecolor(color_map[group])  # 根据EA分配颜色

    plt.title('Comparison of Individual Gains across DE and GA (FF1)', fontsize=16)
    file_name = 'Comparison_of_Individual_Gains_DE_GA_FF1'
    plt.xticks(rotation=45)

    # 添加自定义图例，区分 DE 和 GA
    handles = [
        plt.Line2D([0], [0], color='red', lw=4, label='DE (Differential Evolution)'),
        plt.Line2D([0], [0], color='green', lw=4, label='GA (Genetic Algorithm)'),
    ]
    plt.legend(handles=handles, title="Evolutionary Algorithm", loc="best")  # 使用 loc="best" 来自动选择最佳位置
    
    # 比较相同 Fitness Mode (FF1) 和相同敌人下的不同 EA 的差异
    unique_groups = df['Group'].unique()
    y_max = df['Individual Gain'].max()  # 用于定位 p 值显示位置

    for enemy in enemies:
        group1_label = f"DE E{enemy}"
        group2_label = f"GA E{enemy}"

        if group1_label in unique_groups and group2_label in unique_groups:
            group1 = df[df['Group'] == group1_label]['Individual Gain']
            group2 = df[df['Group'] == group2_label]['Individual Gain']
            
            # 使用 t-test 计算p值
            t_stat, p_value = ttest_ind(group1, group2)

            # 设置两组数据在图中的x轴位置
            x1, x2 = list(unique_groups).index(group1_label), list(unique_groups).index(group2_label)
            y, h = y_max + 10, 5  # 设置 y 轴的显示位置和高度
            
            # 在两组boxplot之间绘制连接线并显示p值
            ax.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.5, color='black')
            ax.text((x1 + x2) * 0.5, y + h, f"p = {p_value:.3f}", ha='center', va='bottom', color='black')

    # 保存图像
    plt.savefig(f'Group_96_plots/{file_name}.png', bbox_inches='tight')
    plt.show()

# 示例调用：绘制带有显著性水平的 boxplot
plot_boxplot_with_significance(all_results, EA_variables)
